# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [33]:
archivos = [ elemento for elemento in carpeta_datos.glob('*/videos/*.json') ]

In [34]:
len(archivos)

237

In [35]:
{archivo.suffix for archivo in archivos }

{'.json'}

In [31]:
archivos[0].suffix

''

- Importar los datos con pandas

In [36]:
import pandas

In [39]:
def safe_df(archivo):
    try:
        df = pandas.read_json(archivo)
    except ValueError as err:
        df = pandas.DataFrame()
        print('Fallo al cargar: ', archivo)
    return df

In [40]:
dataframes = [ safe_df(archivo) for archivo in archivos ]

Fallo al cargar:  datos\pycon-ar-2011\videos\pyconar-2011-alejandro-pereira-silvestre-huens-plone-4-tiempo-de-coccion-30-minutos-pyar.json
Fallo al cargar:  datos\pycon-ar-2011\videos\pyconar-2011-hector-sanchez-gonzalo-odiardmanuel-quinones-entorno-educativo-sugar-pyar.json
Fallo al cargar:  datos\pycon-ar-2011\videos\pyconar-2011-roberto-alsina-manuel-de-la-pena-programando-cross-platform-pyar.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-28-02-00001-720i-fast.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-35-46-00002-720i-fast.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-40-48-00003-720i-fast.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-44-24-00004-720i-fast.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-50-14-00005-720i-fast.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-55-10-00006-720i-fast.json
Fallo al cargar:  datos\pycon-ar-2012\videos\2012-11-16-18-57-42-00007-720i-fast

In [43]:
# %load datos/pycon-ar-2011/videos/pyconar-2011-alejandro-pereira-silvestre-huens-plone-4-tiempo-de-coccion-30-minutos-pyar.json
{
  "description": "Descripci\u00f3n de la charla: Cocci\u00f3n de un sitio Plone completamente funcional con sistema de autenticacion de usuarios, permisos y tipos de contenidos.",
  "language": "spa",
  "recorded": "2011-09-23",
  "speakers": [
    "Alejandro Pereira",
    "Silvestre Huens"
  ],
  "thumbnail_url": "https://i.ytimg.com/vi/UH6WQxQYUb0/hqdefault.jpg",
  "title": "Plone 4: tiempo de cocci\u00f3n 30 minutos",
  "videos": [
    {
      "type": "youtube",
      "url": "https://www.youtube.com/watch?v=UH6WQxQYUb0"
    }
  ]
}


In [46]:
pandas.read_json(archivos[3])

ValueError: arrays must all be same length

- Realizar graficos